In [ ]:
import pandas as pd
import numpy as np
import cv2
import glob
# import imutils
# from imutils import paths
import os
import os.path
%matplotlib inline 
import matplotlib.pyplot as plt
import os
# print(os.listdir("train"))
from keras.preprocessing.image import ImageDataGenerator, array_to_img, img_to_array, load_img



In [ ]:
image = load_img('../input/cap-data/data/validation/QOKZ.png')
# img = cv2.imread(os.path('../input/cap-data/data/validation/QOKZ.png'), cv2.IMREAD_GRAYSCALE)
captcha_image='../input/cap-data/data/validation/QOKZ.png'
gray = cv2.imread(captcha_image)
gray = cv2.cvtColor(gray, cv2.COLOR_BGR2GRAY)

filename = os.path.basename(captcha_image)
print(filename)
captcha_text = os.path.splitext(filename)[0]
print(captcha_text)
thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV, cv2.THRESH_OTSU)[1]
# thresh = cv2.threshold(gray, 127, 255,0,cv2.THRESH_OTSU)[1]
plt.imshow(thresh)

In [ ]:
# creating empty list for holding the coordinates of the letters
letter_image_regions = []
# finding the contours
contours, hierarchy = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
print(len(contours))
# print(len(hierarchy))
for contour in contours:
    # Get the rectangle that contains the contour
    (x, y, w, h) = cv2.boundingRect(contour)
#     print(x,y,w,h)
    if w / h > 1.25:
    # Split it in half into two letter regions
        half_width = int(w / 2)
        letter_image_regions.append((x, y, half_width, h))
        letter_image_regions.append((x + half_width, y, half_width, h))
    else:  
        letter_image_regions.append((x, y, w, h))
# cv2.drawContours(thresh, contours, -1, (0,255,0), 3)
letter_image_regions = sorted(letter_image_regions, key=lambda x: x[0])
print(letter_image_regions)

In [ ]:
# Save each letter as a single image
for letter_bounding_box, letter_text in zip(letter_image_regions, captcha_text):
    # Grab the coordinates of the letter in the image
    x, y, w, h = letter_bounding_box
#     letter_image = gray[y :y + h , x :x + w ]
    letter_image = gray[y - 2:y + h + 2, x - 2:x + w + 2]
    plt.imshow(letter_image)


In [ ]:
SOLVED_CAPTCHA_FOLDER = '../input/cap-data/data/train'
OUTPUT_FOLDER = "../output/train"


# Get the path of all the solved captcha images
solved_captchas = glob.glob(os.path.join(SOLVED_CAPTCHA_FOLDER, "*"))
counts = {}

# loop over the image paths
for (i, captcha) in enumerate(solved_captchas):
    print("processing image {}/{}".format(i + 1, len(solved_captchas)))

    
    # grab the base filename as the text
    filename = os.path.basename(captcha)
    captcha_text = os.path.splitext(filename)[0]

    # Load the image and convert it to grayscale
    image = cv2.imread(captcha)
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    
    # applying threshold
    thresh = cv2.threshold(gray, 127, 255, cv2.THRESH_BINARY_INV, cv2.THRESH_OTSU)[1]

     # finding the contours
    contours, hierarchy = cv2.findContours(thresh.copy(), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # creating empty list for holding the coordinates of the letters
    letter_image_regions = []

    # Now we will loop through each of the contours and extract the letter
    for contour in contours:
        # Get the rectangle that contains the contour
        (x, y, w, h) = cv2.boundingRect(contour)
        
        # checking if any counter is too wide
        # if countour is too wide then there could be two letters joined together or are very close to each other
        if w / h > 1.25:
            # Split it in half into two letter regions
            half_width = int(w / 2)
            letter_image_regions.append((x, y, half_width, h))
            letter_image_regions.append((x + half_width, y, half_width, h))
        else:
            
            letter_image_regions.append((x, y, w, h))
            

    
    # Sort the detected letter images based on the x coordinate to make sure
    # we get them from left-to-right so that we match the right image with the right letter
    letter_image_regions = sorted(letter_image_regions, key=lambda x: x[0])
    
    # Save each letter as a single image
    for letter_bounding_box, letter_text in zip(letter_image_regions, captcha_text):
        # Grab the coordinates of the letter in the image
        x, y, w, h = letter_bounding_box

        # Extract the letter from the original image with a 2-pixel margin around the edge
        letter_image = gray[y - 2:y + h + 2, x - 2:x + w + 2]


        # Get the folder to save the image in
        save_path = os.path.join(OUTPUT_FOLDER, letter_text)

        # creating different output folder for storing different letters
        if not os.path.exists(save_path):
            os.makedirs(save_path)

        # write the letter image to a file
        count = counts.get(letter_text, 1)
        p = os.path.join(save_path, "{}.png".format(str(count)))
        cv2.imwrite(p, letter_image)

        # increment the count
        counts[letter_text] = count + 1


In [ ]:
import imutils
from imutils import paths
letter_folder = '../output/train/A'

#creating empty lists for storing image data and labels
data = []
labels = []
print(len(letter_folder))
for image in paths.list_images(letter_folder):
    img = cv2.imread(image)
#     plt.imshow(img)
#     img = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
    img = cv2.resize(img, (30,30))
    
    # adding a 3rd dimension to the image
    img = np.expand_dims(img, axis = 2)
    
    #grabing the name of the letter based on the folder it is present in
    label = image.split(os.path.sep)[-2]
    
    # appending to the empty lists
    data.append(img)
    labels.append(label)

#converting data and labels to np array
data = np.array(data, dtype = "float")
labels = np.array(labels)
